In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

iris = load_iris()
X = iris['data']
y = iris['target']
#X=X[y!=2]
#y=y[y!=2]
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1
# which is importance for convergence of the neural network
#scaler = StandardScaler()
scaler = MinMaxScaler((-3.142, 3.142))
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(
    X_scaled, y, test_size=0.25, random_state=2)

In [ ]:
import tqdm
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
num_qubits = 4
num_layers = 1
bond = 16

In [ ]:
model_test = VQC(num_qubits,num_layers)
optimizer = torch.optim.Adam(model_test.parameters(), lr = 0.01, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()
EPOCHS = 25
X_train = (torch.from_numpy(X_train_0)).float()
y_train = (torch.from_numpy(y_train_0)).long().reshape(X_train.shape[0],1)
X_test  = Variable(torch.from_numpy(X_test_0)).float()
y_test  = Variable(torch.from_numpy(y_test_0)).long().reshape(X_test.shape[0],1)
model_test.train()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))
accuracy_list_train = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
  l = 0
  c = []
  for b,s in enumerate(X_train):
    psi = create_zero_tensor(num_qubits, bond)
    k1 = F.softmax(model_test(psi,s), dim = 1)
    loss = F.cross_entropy(k1, y_train[b])
    correct = (torch.argmax(k1, dim=1) == y_train[b]).type(torch.FloatTensor)
    c.append(correct.item())
    l+=loss/4
    if int((b+1)%4)==0:
      optimizer.zero_grad()
      l.backward(create_graph = True)
      optimizer.step()
      loss_list[epoch] = l.item()
      loss_epoch = float(l)
      l=torch.zeros(1)
    accuracy_list_train[epoch] = np.array(c).mean()
  with torch.no_grad():
    c=[]
    for j,s in enumerate(X_test):
      psi = create_zero_tensor(num_qubits, bond)
      k1 = F.softmax(model_test(psi,s), dim = 1)
      correct = (torch.argmax(k1, dim=1) == y_test[j]).type(torch.FloatTensor)
      c.append(correct.item())
    accuracy_list[epoch] = np.array(c).mean()
  print("Train Loss: {0:1.4f}; Train Accuracy: {1:1.4f}; Test Accuracy: {2:1.4f}".format(loss_epoch, accuracy_list_train[epoch], accuracy_list[epoch]))
